In [1]:
import pandas as pd

TRAIN = '/kaggle/input/spaceship-titanic/train.csv'

train_df = pd.read_csv(filepath_or_buffer=TRAIN)
# let's do the dumbest thing possible and just replace NaNs with zeros
train_df = train_df.fillna(0)
train_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [2]:
import arrow
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

COLUMNS = ['CryoSleep', 'Age', 'RoomService', 'VIP', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

TARGET = 'Transported'

X_train, X_test, y_train, y_test = train_test_split(train_df[COLUMNS], train_df[TARGET], test_size=0.20, random_state=2024, stratify=train_df[TARGET])

time_start = arrow.now()
regression = LogisticRegression(max_iter=1000, tol=1e-12).fit(X=X_train, y=y_train)
print('model fit in {} iterations took {}'.format(regression.n_iter_[0], arrow.now() - time_start))

print('accuracy: {:5.4f}'.format(accuracy_score(y_true=y_test, y_pred=regression.predict(X=X_test))))
print('model done in {}'.format(arrow.now() - time_start))

model fit in 80 iterations took 0:00:00.121564
accuracy: 0.7734
model done in 0:00:00.132295


In [3]:
from plotly import express

express.histogram(x=COLUMNS, y=regression.coef_[0])

Let's make a submission.

In [4]:
TEST = '/kaggle/input/spaceship-titanic/test.csv'
test_df = pd.read_csv(filepath_or_buffer=TEST).fillna(0)


result_df = pd.DataFrame(data = {'PassengerId' : test_df['PassengerId'].tolist(), 'Transported' : regression.predict(X=test_df[COLUMNS])})
result_df.to_csv(path_or_buf='/kaggle/working/submission.csv', index=False)